In [1]:
from proportion_test import prop_test_1sample as pt1


pt1(9,250,.073,print_out=True)

One sample propotion test
Z porption test p-value 0.033 for real p equals p0.
Binomial test p-value 0.011 for gettig less or equal events if real p equals p0.


{'Z p': 0.03339190637859857, 'Binomial p': 0.01103610978073807}

In [5]:
(1-0.9877406177519058)*2

0.024518764496188394

In [2]:
from scipy.stats import norm
9/250+norm.ppf(.975)*(9/250*(1-9/250)/250)**.5

0.05909233525768833